# Imports

In [ ]:
import subprocess
import json

In [ ]:
def runFullParse (piece:str):
    jsonPath = "outputs/"+piece+".json"
    cmd = ["stack","run","fullParse","--","-n","10",piece, "All"] 
    print("Running command: " + (" ".join(cmd)))
    print("Expecting results in " + jsonPath)
    #res = subprocess.run(cmd, cwd="..")
    res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd="..")

    if res.returncode != 0:
        print("Error in subprocess")
        print(res.stderr)
        print(res.stdout)
        return;
    else:
        f = open(jsonPath)
        results = json.load(f)
        f.close()
        return results
     

In [ ]:
def runFullPieceExperiment(pieces):
    # Runs Heuristic search, Random Search, and Random sample search on the given piece.
    # Returns the scores for each 
    res = []
    for piece in pieces:
      res.append(runFullParse(piece))
    return res 



# Run Experiments

In [ ]:
res = runFullPieceExperiment(pieces[0:3])